In [1]:
import pandas as pd
from contractions import contractions_dict

!pip install simpletransformers
!pip install transformers
!pip install tensorboardx

In [2]:
news_df=pd.read_csv('news.csv')

In [3]:
news_df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [4]:
news_df['prediction_label']=0
news_df.loc[news_df['label']=='FAKE', 'prediction_label']=1

In [5]:
news_df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
news_df['text']=news_df['text'].apply(lambda x: x.lower())

In [7]:
news_df.head()

,title,text,label,prediction_label
0,You Can Smell Hillary’s Fear,"daniel greenfield, a shillman journalism fello...",FAKE,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1
2,Kerry to go to Paris in gesture of sympathy,u.s. secretary of state john f. kerry said mon...,REAL,0
3,Bernie supporters on Twitter erupt in anger ag...,"— kaydee king (@kaydeeking) november 9, 2016 t...",FAKE,1
4,The Battle of New York: Why This Primary Matters,it's primary day in new york and front-runners...,REAL,0


In [8]:
news_df.drop(columns=['title', 'label'], inplace=True)

In [9]:
news_df.rename(columns={'prediction_label':'label'}, inplace=True)

In [10]:
news_df.head()

,text,label
0,"daniel greenfield, a shillman journalism fello...",1
1,google pinterest digg linkedin reddit stumbleu...,1
2,u.s. secretary of state john f. kerry said mon...,0
3,"— kaydee king (@kaydeeking) november 9, 2016 t...",1
4,it's primary day in new york and front-runners...,0


In [11]:
def expand_contractions(text):
    for i in text:
        if i in contractions_dict.keys():
            i=re.sub(i, contractions_dict[i], i)
    return text

news_df['text']=news_df['text'].apply(lambda x: expand_contractions(x))

In [14]:
from sklearn.model_selection import train_test_split

train, test=train_test_split(news_df, test_size=0.3, stratify=news_df['label'], random_state=42)

In [16]:
from simpletransformers.classification import ClassificationModel

bertModel=ClassificationModel('bert', 'bert-base-uncased', use_cuda=False,
                         args={
                             'num_train_epochs':1,
                             'learning_rate':5e-5,
                             'max_seq_length': 128,
                             'overwrite_output_dir': True,
                             'train_batch_size': 8,
                             'eval_batch_size': 8
                         })

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
bertModel.train_model(train)

In [18]:
result, model_outputs, wrong_predictions=bertModel.eval_model(test)

/Users/dkaranikos/opt/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:965: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


In [19]:
result

{'mcc': 0.9466692033479123,
 'tp': 911,
 'tn': 939,
 'fp': 13,
 'fn': 38,
 'eval_loss': 0.10117542303530225}

In [15]:
robertaModel=ClassificationModel('roberta', 'roberta-base', use_cuda=False,
                         args={
                             'num_train_epochs':1,
                             'learning_rate':5e-5,
                             'max_seq_length': 128,
                             'overwrite_output_dir': True,
                             'train_batch_size': 8,
                             'eval_batch_size': 8,
                             'sliding_window': True,
                             'wandb_project': 'visualizations'
                         })


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [17]:
robertaModel.train_model(train)

/Users/dkaranikos/opt/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:386: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


wandb: Currently logged in as: aud-z (use `wandb login --relogin` to force relogin)


/Users/dkaranikos/opt/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(6067, 0.1256677008762772)

In [18]:
result, model_outputs, wrong_predictions=robertaModel.eval_model(test)

/Users/dkaranikos/opt/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:965: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Training loss,0.01167
lr,0.0
global_step,6050
_step,120
_runtime,39870
_timestamp,1607484934


Training loss,▇█▁▁▁▅▂▁▁▁▁▁▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁█▁▁▁▁▁▁▁
lr,▂▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [19]:
result

{'mcc': 0.9842844227122913,
 'tp': 936,
 'tn': 950,
 'fp': 2,
 'fn': 13,
 'eval_loss': 0.08020694609767666}